##### Copyright 2021 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# ポリシー

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/agents/tutorials/3_policies_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> TensorFlow.orgで表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/agents/tutorials/3_policies_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colabで実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/agents/tutorials/3_policies_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示{</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/agents/tutorials/3_policies_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード/a0}</a></td>
</table>

## はじめに

強化学習では、ポリシーは環境からの観察を行動または行動の分布にマッピングします。TF-Agent では、環境からの観測は、名前付きタプル`TimeStep('step_type', 'discount', 'reward', 'observation')`に含まれています。また、ポリシーはタイムステップを行動または行動の分布にマップします。ほとんどのポリシーは`timestep.observation`を使用し、一部のポリシーは`timestep.step_type`を使用します（ステートフルポリシーのエピソードの開始時に状態をリセットする場合など）。`timestep.discount`および`timestep.reward`は通常無視されます。

ポリシーは、TF-Agent の他のコンポーネントと次のように関連しています。ほとんどのポリシーには、行動や TimeStep からの行動の分布を計算するニューラルネットワークがあります。エージェントには、デプロイするためにトレーニングされた主要なポリシーやデータ収集のためのノイジーなポリシーなど、さまざまな目的のために複数のポリシーを含めることができます。ポリシーは保存/復元でき、エージェントとは関係なくデータ収集、評価などに使用できます。

Tensorflow で記述しやすいポリシーもあれば（ニューラルネットワークを使用するポリシーなど）、Python で記述しやすいポリシーもあります（行動のスクリプトに従う場合など）。そのため、TF Agent では、Python と Tensorflow の両方のポリシーが許可されています。さらに、TensorFlow で記述されたポリシーを Python 環境で使用する必要がある場合や、逆にトレーニングに TensorFlow ポリシーを使用し、後で Python の本番環境にデプロイすることが必要な場合もあります。これを簡単に実行するために、Python と TensorFlow ポリシー間を変換するラッパーが提供されています。

ポリシーのもう 1 つの興味深いクラスは、特定のポリシーを特定の方法で変更するポリシーラッパーです。例えば特定のタイプのノイズを追加し、確率的ポリシーの greedy または epsilon-greedy バージョンを作成したり、複数のポリシーをランダムに混合できます。 

## セットアップ

TF-agent をまだインストールしていない場合は、次を実行します。

In [ ]:
!pip install tf-agents

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

from tf_agents.specs import array_spec
from tf_agents.specs import tensor_spec
from tf_agents.networks import network

from tf_agents.policies import py_policy
from tf_agents.policies import random_py_policy
from tf_agents.policies import scripted_py_policy

from tf_agents.policies import tf_policy
from tf_agents.policies import random_tf_policy
from tf_agents.policies import actor_policy
from tf_agents.policies import q_policy
from tf_agents.policies import greedy_policy

from tf_agents.trajectories import time_step as ts

## Python ポリシー

Python ポリシーのインターフェースは、`policies/py_policy.PyPolicy`で定義されています。主なメソッドは次のとおりです。


In [ ]:
class Base(object):

  @abc.abstractmethod
  def __init__(self, time_step_spec, action_spec, policy_state_spec=()):
    self._time_step_spec = time_step_spec
    self._action_spec = action_spec
    self._policy_state_spec = policy_state_spec

  @abc.abstractmethod
  def reset(self, policy_state=()):
    # return initial_policy_state.
    pass

  @abc.abstractmethod
  def action(self, time_step, policy_state=()):
    # return a PolicyStep(action, state, info) named tuple.
    pass

  @abc.abstractmethod
  def distribution(self, time_step, policy_state=()):
    # Not implemented in python, only for TF policies.
    pass

  @abc.abstractmethod
  def update(self, policy):
    # update self to be similar to the input `policy`.
    pass

  @property
  def time_step_spec(self):
    return self._time_step_spec

  @property
  def action_spec(self):
    return self._action_spec

  @property
  def policy_state_spec(self):
    return self._policy_state_spec

最も重要なメソッドは`action(time_step)`で、環境からの観測を含む`time_step`を、次の属性を含む PolicyStep という名前のタプルにマップします。

- `action`：環境に適用される行動。
- `state`：次の行動の呼び出しにフィードされるポリシーの状態（RNN 状態など）。
- `info`：行動のログ確率などのオプションの補足情報。

`time_step_spec`および`action_spec`は、入力されたタイムステップと出力された行動の仕様です。ポリシーには`reset`関数もあり、通常、ステートフルポリシーの状態をリセットするために使用されます。`update(new_policy)`関数は`self`を`new_policy`に向けて更新します。

それでは、Python ポリシーの例をいくつか見てみましょう。


### 例1：ランダムPythonポリシー

`PyPolicy`の簡単な例は、`RandomPyPolicy`で、指定された個別/連続する action_spec に対してランダムな行動を生成します。入力`time_step`は無視されます。

In [ ]:
action_spec = array_spec.BoundedArraySpec((2,), np.int32, -10, 10)
my_random_py_policy = random_py_policy.RandomPyPolicy(time_step_spec=None,
    action_spec=action_spec)
time_step = None
action_step = my_random_py_policy.action(time_step)
print(action_step)
action_step = my_random_py_policy.action(time_step)
print(action_step)

### 例2：スクリプト化されたPythonポリシー

スクリプト化されたポリシーは、`(num_repeats, action)`タプルのリストとして表される行動のスクリプトを再生します。`action`関数が呼び出されるたびに、指定された繰り返し回数が完了するまでリストから次の行動が返され、リスト内の次の行動に移動します。リストの先頭から実行するには`reset`メソッドを呼び出します。

In [ ]:
action_spec = array_spec.BoundedArraySpec((2,), np.int32, -10, 10)
action_script = [(1, np.array([5, 2], dtype=np.int32)), 
                 (0, np.array([0, 0], dtype=np.int32)), # Setting `num_repeats` to 0 will skip this action.
                 (2, np.array([1, 2], dtype=np.int32)), 
                 (1, np.array([3, 4], dtype=np.int32))]

my_scripted_py_policy = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=None, action_spec=action_spec, action_script=action_script)

policy_state = my_scripted_py_policy.get_initial_state()
time_step = None
print('Executing scripted policy...')
action_step = my_scripted_py_policy.action(time_step, policy_state)
print(action_step)
action_step= my_scripted_py_policy.action(time_step, action_step.state)
print(action_step)
action_step = my_scripted_py_policy.action(time_step, action_step.state)
print(action_step)

print('Resetting my_scripted_py_policy...')
policy_state = my_scripted_py_policy.get_initial_state()
action_step = my_scripted_py_policy.action(time_step, policy_state)
print(action_step)

## TensorFlow ポリシー

TensorFlow ポリシーは、Python ポリシーと同じインターフェースに従います。いくつかの例を見てみましょう。

### 例1：ランダム TF ポリシー

RandomTFPolicy を使用すると、指定された離散/連続`action_spec`に従ってランダムな行動を生成できます。入力`time_step`は、無視されます。


In [ ]:
action_spec = tensor_spec.BoundedTensorSpec(
    (2,), tf.float32, minimum=-1, maximum=3)
input_tensor_spec = tensor_spec.TensorSpec((2,), tf.float32)
time_step_spec = ts.time_step_spec(input_tensor_spec)

my_random_tf_policy = random_tf_policy.RandomTFPolicy(
    action_spec=action_spec, time_step_spec=time_step_spec)
observation = tf.ones(time_step_spec.observation.shape)
time_step = ts.restart(observation)
action_step = my_random_tf_policy.action(time_step)

print('Action:')
print(action_step.action)

### 例2：アクターポリシー

アクターポリシーは、`time_steps`を行動にマップするネットワーク、または、`time_steps`を行動上の分布にマップするネットワークのいずれかを使用して作成できます。


#### 行動ネットワークの使用

次のようにネットワークを定義します。

In [ ]:
class ActionNet(network.Network):

  def __init__(self, input_tensor_spec, output_tensor_spec):
    super(ActionNet, self).__init__(
        input_tensor_spec=input_tensor_spec,
        state_spec=(),
        name='ActionNet')
    self._output_tensor_spec = output_tensor_spec
    self._sub_layers = [
        tf.keras.layers.Dense(
            action_spec.shape.num_elements(), activation=tf.nn.tanh),
    ]

  def call(self, observations, step_type, network_state):
    del step_type

    output = tf.cast(observations, dtype=tf.float32)
    for layer in self._sub_layers:
      output = layer(output)
    actions = tf.reshape(output, [-1] + self._output_tensor_spec.shape.as_list())

    # Scale and shift actions to the correct range if necessary.
    return actions, network_state

TensorFlow では、ほとんどのネットワークレイヤーがバッチ処理用に設計されているため、入力 time_steps がバッチ処理され、ネットワークの出力もバッチ処理されることが想定されています。また、ネットワークは、指定された action_spec の正常な範囲で行動を生成する役割を担います。これは慣習的に[-1, 1] で行動を生成する最終レイヤーに tanh アクティベーションなどを使用し、それを入力 action_spec として正常な範囲にスケーリングとシフトを行うことで実現されます。（`tf_agents/agents/ddpg/networks.actor_network()`を参照してください。）

上記のネットワークを使用してアクターポリシーを作成できます。

In [ ]:
input_tensor_spec = tensor_spec.TensorSpec((4,), tf.float32)
time_step_spec = ts.time_step_spec(input_tensor_spec)
action_spec = tensor_spec.BoundedTensorSpec((3,),
                                            tf.float32,
                                            minimum=-1,
                                            maximum=1)

action_net = ActionNet(input_tensor_spec, action_spec)

my_actor_policy = actor_policy.ActorPolicy(
    time_step_spec=time_step_spec,
    action_spec=action_spec,
    actor_network=action_net)

これは、time_step_spec に続く time_steps のバッチに適用できます。

In [ ]:
batch_size = 2
observations = tf.ones([2] + time_step_spec.observation.shape.as_list())

time_step = ts.restart(observations, batch_size)

action_step = my_actor_policy.action(time_step)
print('Action:')
print(action_step.action)

distribution_step = my_actor_policy.distribution(time_step)
print('Action distribution:')
print(distribution_step.action)

上記の例では、行動テンソルを生成する行動ネットワークを使用してポリシーを作成しました。この場合、`policy.distribution(time_step)`は、`policy.action(time_step)`の出力に関する確定的（デルタ）分布です。確率的ポリシーを作成するには、行動にノイズを追加するポリシーラッパーでアクターポリシーをラップします。また、以下のように、行動ネットワークの代わりに行動分布ネットワークを使用してアクターポリシーを作成することもできます。

#### 行動分布ネットワークの使用

In [ ]:
class ActionDistributionNet(ActionNet):

  def call(self, observations, step_type, network_state):
    action_means, network_state = super(ActionDistributionNet, self).call(
        observations, step_type, network_state)

    action_std = tf.ones_like(action_means)
    return tfp.distributions.MultivariateNormalDiag(action_means, action_std), network_state


action_distribution_net = ActionDistributionNet(input_tensor_spec, action_spec)

my_actor_policy = actor_policy.ActorPolicy(
    time_step_spec=time_step_spec,
    action_spec=action_spec,
    actor_network=action_distribution_net)

action_step = my_actor_policy.action(time_step)
print('Action:')
print(action_step.action)
distribution_step = my_actor_policy.distribution(time_step)
print('Action distribution:')
print(distribution_step.action)

上記では、行動は指定された行動仕様[-1, 1]の範囲にクリップされていることに注意してください。これは、デフォルトで ActorPolicy clip=True のコンストラクタ引数があるためです。これをfalseに設定すると、ネットワークにより生成されたクリップされていない行動が返されます。 

たとえば、`stochastic_policy.distribution().mode()`を行動として選択する GreedyPolicy ラッパーと、この greedy な行動に関する確定的/デルタ分布をその`distribution()`として使用することにより、確率的ポリシーは、確定的ポリシーに変換できます。

### 例3：Q ポリシー

Q ポリシーは、DQN などのエージェントで使用されるポリシーで、個別の行動ごとにQ値を予測する Q ネットワークに基づいています。特定のタイムステップで、Q ポリシーの行動分布は q 値をロジットとして使用して作成されたカテゴリカル分布です。


In [ ]:
input_tensor_spec = tensor_spec.TensorSpec((4,), tf.float32)
time_step_spec = ts.time_step_spec(input_tensor_spec)
action_spec = tensor_spec.BoundedTensorSpec((),
                                            tf.int32,
                                            minimum=0,
                                            maximum=2)
num_actions = action_spec.maximum - action_spec.minimum + 1


class QNetwork(network.Network):

  def __init__(self, input_tensor_spec, action_spec, num_actions=num_actions, name=None):
    super(QNetwork, self).__init__(
        input_tensor_spec=input_tensor_spec,
        state_spec=(),
        name=name)
    self._sub_layers = [
        tf.keras.layers.Dense(num_actions),
    ]

  def call(self, inputs, step_type=None, network_state=()):
    del step_type
    inputs = tf.cast(inputs, tf.float32)
    for layer in self._sub_layers:
      inputs = layer(inputs)
    return inputs, network_state


batch_size = 2
observation = tf.ones([batch_size] + time_step_spec.observation.shape.as_list())
time_steps = ts.restart(observation, batch_size=batch_size)

my_q_network = QNetwork(
    input_tensor_spec=input_tensor_spec,
    action_spec=action_spec)
my_q_policy = q_policy.QPolicy(
    time_step_spec, action_spec, q_network=my_q_network)
action_step = my_q_policy.action(time_steps)
distribution_step = my_q_policy.distribution(time_steps)

print('Action:')
print(action_step.action)

print('Action distribution:')
print(distribution_step.action)

## ポリシーラッパー

ポリシーラッパーは、特定のポリシーをラップして変更するために使用できます（ノイズを追加する場合など）。ポリシーラッパーはポリシー（Python / TensorFlow）のサブクラスであるため、他のポリシーと同じように使用できます。 

### 例：Greedy ポリシー

greedyラッパーは、`distribution()`を実装する TensorFlow ポリシーをラップするために使用できます。`GreedyPolicy.action()`は`wrapped_policy.distribution().mode()`を返し、`GreedyPolicy.distribution()`は`GreedyPolicy.action()`の確定的/デルタ分布です。

In [ ]:
my_greedy_policy = greedy_policy.GreedyPolicy(my_q_policy)

action_step = my_greedy_policy.action(time_steps)
print('Action:')
print(action_step.action)

distribution_step = my_greedy_policy.distribution(time_steps)
print('Action distribution:')
print(distribution_step.action)